In [ ]:
!pip install requests
!pip install bs4
!pip install pandas
!pip install lxml
!pip install numpy

In [ ]:
from bs4 import BeautifulSoup as bs#HTML parser 구문분석
import time
import csv
import matplotlib.pyplot as plt
from urllib.request import urlopen
import pandas as pd
import numpy as np

In [ ]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}
data = requests.get('https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&date=20230127', headers = headers) #, headers = headers
soup = BeautifulSoup(data.text, 'html.parser')

title = soup.select_one('#old_content > table > tbody > tr > td.title > div > a')
print(title.text)
print(title['href'])

In [ ]:
# trs = soup.select('#old_content > table > tbody > tr')

# for tr in trs:
#   print(tr)
trs = soup.select('#old_content > table > tbody > tr')

for tr in trs:
  a_tag = tr.select_one('td.title > div > a')
  if a_tag is not None:
    rank = tr.select_one('td > img')['alt']
    title = a_tag.text
    star = tr.select_one('td.point').text
    print(rank, title, star)

In [ ]:
url = 'https://movie.naver.com/movie/point/af/list.naver'
webpage = urlopen(url).read().decode()
webpage

In [ ]:
naver = bs(webpage, 'html.parser')
type(naver)

In [ ]:
trs = naver.select('#old_content > table tr')[1:]
trs[0]

In [ ]:
df = pd.DataFrame(columns={'col1','col2','col3','col4','col5','col6'})

for item in trs :
  It = []

  #파싱
  tds = item.find_all('td')
  
  no = tds[0].text
  writer = tds[1].find('a').text
  em = tds[1].find('em').text
  netizen = tds[1].text.split('\n')[5:-2]
  netizen = ' '.join(netizen).strip()
  author = tds[2].text

  #리스트에 추가
  It.append(no)
  It.append(writer)
  It.append(em)
  It.append(author[:-8])
  It.append(author[-8:])

  #데이터 프레임에 추가
  # (인덱스 값 만들기)
  idx = df.index.max()
  if np.isnan(idx) : idx = 0
  else : idx = idx + 1

  #데이터 프레임에 추가
  df.loc[idx] = It
  print(It)




In [ ]:
df.columns = ['번호', '영화명', '평점', '리뷰', '작성자아이디', '작성일']
df

In [ ]:
df['평점'] = df['평점'].astype('int')

In [ ]:
dfg = df.groupby('영화명').mean()
dfg

In [ ]:
dfg.sort_values('평점').plot(kind='barh')
plt.show()